In [ ]:
from IPython.core.magics.execution import _format_time as format_time
from IPython.display import Javascript

Javascript('''
function swap(el, src, t) {
  console.log("swap", el, src, t);
  var old = el.src;
  el.src = src;
  setTimeout(function() {el.src = old;}, t);
}

sieve.onclick = function() {
  swap(document.getElementById("sieve"), 'sieve.gif', 37000);
};
''')

# Optimised Primes

Emlyn Corrin

<img data-gifffer="sieve.gif" />

![](prime.png)
<!--- Image (public domain) from:
https://www.flickr.com/photos/114305749@N08/24438440681
-->

## Why?

- Because it's fun?
- Mathematical or programming exercise
- Online programming contests (Project Euler etc.)

## What is a prime?

A prime number (or a prime) is a natural number greater than 1 that has no positive divisors other than 1 and itself.  
<div style="text-align: right">&mdash; Wikipedia</div>

In [ ]:
def is_prime(n):
    # Greater than 1
    if n <= 1:
        return False
    # Any positive divisors (> 1 and < n)?
    for i in range(2, n):
        if n % i == 0:
            return False
    # If not, it must be a prime
    return True

## Let's generate a few more

In [ ]:
[i for i in range(10) if is_prime(i)]

In [ ]:
print(', '.join(str(i) for i in range(2000) if is_prime(i)))

## What about generating them on demand

In [ ]:
from itertools import count

def primes(limit):
    n = 0
    for i in count():
        if is_prime(i):
            yield i
            n += 1
            if n >= limit:
                break
    
list(primes(10))

## But how fast is it?

In [ ]:
import time

def test(limit):
    t0 = time.perf_counter()
    numbers = list(primes(limit))
    t = time.perf_counter() - t0
    print("Took: {}".format(format_time(t)))
    print("First 20 primes: {}".format(", ".join(map(str, numbers[:20]))))
    print("{0}th prime is {1}".format(len(numbers), numbers[-1]))

In [ ]:
test(1000)

## How can we make it faster?

### What about skipping all even numbers (apart from 2)?

In [ ]:
def primes(limit):
    def is_prime(n):
        for i in range(3, n, 2):
            if n % i == 0:
                return False
        return True
    n = 0
    yield 2
    for i in count(3, 2):
        if is_prime(i):
            yield i
            n += 1
            if n >= limit:
                break

### How much faster is it?

In [ ]:
test(1000)

## Can we reduce the number of checks further?

Imagine we have a candidate prime $n$, and find a factor $f$.  
We must also have another factor $n/f$, where one is $\leq \sqrt n$ and the other is $\geq \sqrt n$.
So we only have to check up to $\sqrt n$

In [ ]:
from math import sqrt

def primes(limit):
    def is_prime(n):
        for i in range(3, int(sqrt(n)) + 1, 2):
            if n % i == 0:
                return False
        return True
    n = 0
    yield 2
    for i in count(3, 2):
        if is_prime(i):
            yield i
            n += 1
            if n >= limit:
                break

## How much faster is this?

In [ ]:
test(1000)

## The sieve of Eratosthenes

1. start with a grid of numbers, from 2 to max_prime
2. find first (next) unmarked number, return that as a prime
3. mark all multiples of it (actually just from n² onwards)
4. go back to 2.
<img src="sieve.png" id="sieve" />

In [ ]:
def primes(limit):
    sieve = [True] * limit
    for i in range(2, limit):
        if sieve[i]:
            yield i
            for j in range(i, limit, i):
                sieve[j] = False

In [ ]:
def primes(limit):
    sieve = [True] * (limit // 2)
    yield 2
    for i in range(3, limit, 2):
        if sieve[i//2]:
            yield i
            for j in range(i*i, limit, i*2):
                sieve[j//2] = False

In [ ]:
test(10000000)

## Problems?

### Memory use
Can be mitigated somewhat:
- Use packed data structure (e.g. struct module), 1 byte per cell
- Even encode 8 cells into 1 byte (1 bit each)
- No need for cells for even numbers
- Or even multiples of 3 (only check numbers of form $6n \pm 1$)

### Need to allocate storage upfront
Sometimes we don't know in advance how much to allocate
(e.g. first 100k primes, find first prime with X property...)

## What about storing a list of primes so far, and only test dividing by those?

In [ ]:
def primes(limit):
    primelist = [2]
    yield 2
    for candidate in range(3, limit + 1, 2):
        isprime = True
        for p in primelist:
            if candidate % p == 0:
                isprime = False
                break
        if isprime:
            yield candidate
            primelist.append(candidate)

In [ ]:
test(100000)

## Better?
Now don’t have to decide upper limit in advance, but slower

What about switching things around… for each prime store the next multiple higher than candidate, then we just just have to check if candidate is in the list, not multiple test divisions per candidate.
For each multiple in the list, we store the original prime, so that when we reach it, we we can add it to generate the next multiple. But it could be a multiple of more than one prime, so we have to store a list of source primes:

In [ ]:
def primes(limit):
    state = {}
    for candidate in range(2, limit + 1):
        if candidate in state:
            for factor in state[candidate]:
                if candidate + factor in state:
                    state[candidate + factor].append(factor)
                else:
                    state[candidate + factor] = [factor]
            del state[candidate]
        else:
            yield candidate
            state[2 * candidate] = [candidate]

In [ ]:
test(1000000)

We can make a few optimisations:
Defaultdict so we don’t have to check if a number is present
We skip even numbers, and therefore even multiples of primes
When we find a prime, p, the first multiple we have to add to the state is p^2, because smaller multiples will have another factor less than p  (p*q, where q < p).


In [ ]:
from collections import defaultdict

def primes(limit):
    yield 2
    state = defaultdict(list)
    for candidate in range(3, limit + 1, 2):
        if candidate in state:
            for factor in state[candidate]:
                state[candidate + 2 * factor].append(factor)
            del state[candidate]
        else:
            yield candidate
            state[candidate * candidate] = [candidate]

In [ ]:
test(1000000)